In [ ]:
from array import *
from IPython.core.application import List
import itertools
import math
import time
import numpy as np
import pylab as pl
import networkx as nx
import random
from numpy import random

In [ ]:
class MEC:
    def __init__(self, mname, pcrate, res, ids):
        self.name = mname
        self.processorRate = pcrate
        self.nores = res
        self.id = ids

    def seeContent(self):
            print(f"MEC name : {self.name} Processor Rate : {self.processorRate} Number of Resources : {self.nores} Id : {self.id}")

In [ ]:
class SFC:
    def __init__(self,lat):
        self.VNF = [] 
        self.latency = lat

    def add_VNF(self,vnf):
        self.VNF.append(vnf)
         #print(self.VNF)
        
    def viewSF(self):
        print(f"{self.VNF}, {self.latency}")

In [ ]:
m = []
m.append(MEC('m1',7,12,101))
m.append(MEC('m2',9,8,103))
m.append(MEC('m3',6,9,102))
m.append(MEC('m4',10,5,104))
m.append(MEC('m5',7,5,105))

#m4 = MEC('m4',20,12,101)
for i in range(len(m)):
  m[i].seeContent()

print(m[0].name)
print(m[1].name)
print(m[2].name)
print(m[3].name)
# m4.seeContent()

MEC name : m1 Processor Rate : 7 Number of Resources : 12 Id : 101
MEC name : m2 Processor Rate : 9 Number of Resources : 8 Id : 103
MEC name : m3 Processor Rate : 6 Number of Resources : 9 Id : 102
MEC name : m4 Processor Rate : 10 Number of Resources : 5 Id : 104
MEC name : m5 Processor Rate : 7 Number of Resources : 5 Id : 105
m1
m2
m3
m4


In [ ]:
s1 = SFC(5)
s1.add_VNF("v2")
s1.add_VNF("v5")
s1.add_VNF("v7")
s1.add_VNF("v3")
s1.add_VNF("v1")
s1.add_VNF("v3")
s1.viewSF()

['v2', 'v5', 'v7', 'v3', 'v1', 'v3'], 5


In [ ]:
allocations=[['v3','v2','v1','v5'],['v3','v5','v4'],['v4','v2','v1','v7'],['v6','v5','v1','v3'],['v7','v9','v4','v6','v1']]

In [ ]:
for i in allocations: # Outer loop  
    for j in i: # inner loop  
        print(j, end = " ") # print inserted elements.
    print('')  

v3 v2 v1 v5 
v3 v5 v4 
v4 v2 v1 v7 
v6 v5 v1 v3 
v7 v9 v4 v6 v1 


In [ ]:
len(s1.VNF)
len(m)

5

In [ ]:
#mec_count=int(input('No of MECs '))
out = []  # initially output matrix is empty  
for i in range(len(s1.VNF)): # iterate to the end of rows  
    print(s1.VNF[i]) 
    row=[] 
    for j in range(len(m)): # j iterate to the end of column 
      for k in range(len(allocations[j])):
        if(s1.VNF[i]==allocations[j][k]):
          print(m[j].name, end = " ")
          row.append(m[j].name)
    out.append(row) 
    print(" ")
print(out)

v2
m1 m3  
v5
m1 m2 m4  
v7
m3 m5  
v3
m1 m2 m4  
v1
m1 m3 m4 m5  
v3
m1 m2 m4  
[['m1', 'm3'], ['m1', 'm2', 'm4'], ['m3', 'm5'], ['m1', 'm2', 'm4'], ['m1', 'm3', 'm4', 'm5'], ['m1', 'm2', 'm4']]


In [ ]:
# if(out[0][0]=='m1'):
#   exec("%s = %d" % (out[0][0]))
#   print(out[0][0].processorDelay)
len(out[0])

2

In [ ]:
def poissonDist():
  x = random.poisson(lam=5)           #MEAN NUMBER OF PACKETS FOR CALCULATION CURRENT LOAD ON THE MEC
  return x


In [ ]:
def reward():           #REWARD MATRIX 
  #R=[[]]
  # dummy = list(itertools.repeat(0, len(m)))
  # for i in range(len(s1.VNF)):
  #     R.append(dummy)
  R = [ [0]*(len(m)) for i in range(len(s1.VNF))]
  for i in range(len(s1.VNF)):
    for j in range(len(out[i])):
      s=out[i][j]
      num = int(s[1:])
      # print(m[j].name,'for v',i)
      curload = poissonDist()
      # if(m[num-1].name == "m1"):             # CHECK FOR OVERLOAD
      #  curload = 15
      pd=(curload+1)/m[num-1].processorRate
      R[i][num-1]=(s1.latency-pd)**(11/9)                 #R[i][num-1]=(s1.latency-m[num-1].processorDelay)**(11/9)  
  #print("Reward matrix for SFC1 request:\n")
  return(R)


In [ ]:
#Rew=reward()
rew=reward()

In [ ]:
exception=0
def randomSample(lst, exception):     #RANDOM FUNCTION FOR CHOSING ACTION OF THE STATE     
    while True:
        choice = random.choice(lst)
        if choice != exception:
            return choice

In [ ]:
# s = "m2"
# num = s[1:]  # slice the string from index 1 to the end
# print(num)
randomSample(rew[0], exception)

5.4431600006975085

In [ ]:
alpha=0.01      #LEARNING RATE
disrate=0.8

In [ ]:
Q = [ [0]*(len(m)) for i in range(len(s1.VNF))]

In [ ]:
Q= np.zeros((len(s1.VNF),len(m)))
print("Query matrix\n",Q)
def queryMatrixComputation():
 for j in range(100):      # ENTER EPOCH HERE
   
   print("Query table for epoch",j+1)
   #print(rew)
   #print("")
   for i in range(len(s1.VNF)):
     rew=reward()
     ran=randomSample(rew[i], exception)   # random selection of action
     #print(ran,rew[i].index(ran))
     if(i<len(s1.VNF)):
       maxim = max(rew[i])            
       w=rew[i].index(ran)
       Q[i][rew[i].index(ran)]=Q[i][rew[i].index(ran)] + alpha*(rew[i][rew[i].index(ran)] + disrate*maxim - Q[i][rew[i].index(ran)])
      #  if(Qcalculation>Q[i][rew[i].index(ran)]):
      #   Q[i][rew[i].index(ran)] = Qcalculation
   print(Q)
   print("")
      # print(maxim)
 return(Q)

Query matrix
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [ ]:
start_time = time.time()
Q=queryMatrixComputation()
print("")
Q
print('--- %s seconds ---' % round(time.time() - start_time,7))


Query table for epoch 1
[[0.09783816 0.         0.         0.         0.        ]
 [0.10396144 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.11534603]
 [0.1030379  0.         0.         0.         0.        ]
 [0.10884654 0.         0.         0.         0.        ]
 [0.         0.11507419 0.         0.         0.        ]]

Query table for epoch 2
[[0.19483665 0.         0.         0.         0.        ]
 [0.10396144 0.         0.         0.11072686 0.        ]
 [0.         0.         0.         0.         0.21216945]
 [0.20338237 0.         0.         0.         0.        ]
 [0.10884654 0.         0.1047052  0.         0.        ]
 [0.         0.11507419 0.         0.11314708 0.        ]]

Query table for epoch 3
[[0.29515875 0.         0.         0.         0.        ]
 [0.20664374 0.         0.         0.11072686 0.        ]
 [0.         0.         0.         0.         0.32100363]
 [0.20338237 0.11828984 0.         0.         0.      

In [ ]:
#Q[i][rew[i].index(ran)] = Q[i][rew[i].index(ran)] + alpha(rew[i][rew[i].index(ran)] + disrate*maxim - Q[i][rew[i].index(ran)])

In [ ]:
# import numpy as np

def optimalChain():         #SELCT MAX OF ALL THE VALUES(EXPLOITATION)
  opti_chain=[]
  for i in range(len(Q)):
    mm=max(Q[i])
    no = [index for (index, item) in enumerate(Q[i]) if item == mm]
    print(mm,no[0]+1)
    opti_chain.append(m[no[0]].name)
  print('')
  return(opti_chain)


In [ ]:
result=optimalChain()
print("THE OPTIMIZED SERVICE FUNCTION CHAIN")
result

4.497380636861189 1
3.3276926760705168 4
4.560816458919681 5
3.3518767541144543 4
3.1640246884788468 4
3.3124230929329532 4

THE OPTIMIZED SERVICE FUNCTION CHAIN


['m1', 'm4', 'm5', 'm4', 'm4', 'm4']